In [2]:
from cogdl.datasets.grb_data import Cora_GRBDataset
from cogdl.utils import set_random_seed
from cogdl.utils.grb_utils import evaluate
import copy
import torch
dataset = Cora_GRBDataset()
graph = copy.deepcopy(dataset.get(0))
# device = "cpu"
device = "cuda:0"
device_ids = [0]
graph.to(device)
test_mask = graph.test_mask
set_random_seed(40)

ModuleNotFoundError: No module named 'cogdl'

In [3]:
# train surrogate model
from cogdl.models.nn import GCN
from cogdl.trainer import Trainer
from cogdl.wrappers import fetch_model_wrapper, fetch_data_wrapper
model = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=2,
    dropout=0.5,
    activation=None
)
print(model)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper1 = mw_class(model, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=[0])
trainer.run(model_wrapper1, dataset_wrapper)
model.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model.to(device)
test_score = evaluate(model,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for surrogate model: {:.4f}.".format(test_score))

GCN(
  (layers): ModuleList(
    (0): GCNLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)
Model Parameters: 19847


Epoch: 70, train_loss:  2.6498, val_acc:  0.7463:  35%|███▌      | 70/200 [00:01<00:02, 60.44it/s] 


Saving 21-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.7885572139303483, 'val_acc': 0.8022388059701493}
Test score before attack for surrogate model: 0.7886.


In [4]:
# train target model

model_target = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=3,
    dropout=0.5,
    activation="relu"
)
print(model_target)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score = evaluate(model_target,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score))

GCN(
  (layers): ModuleList(
    (0): GCNLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (act): ReLU(inplace=True)
    )
    (1): GCNLayer(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (act): ReLU(inplace=True)
    )
    (2): GCNLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)
Model Parameters: 24007


Epoch: 200, train_loss:  0.7375, val_acc:  0.7388: 100%|██████████| 200/200 [00:04<00:00, 46.02it/s]


Saving 178-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.7910447761194029, 'val_acc': 0.7873134328358209}
Test score before attack for target model: 0.7910.


In [11]:
from cogdl.attack.modification import DICE
n_mod_ratio = 0.3
ratio_delete = 0.6
attack = DICE(int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio),
              ratio_delete,
              device=device)
graph_attack = attack.attack(graph)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

Delete internally......


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

: 

In [6]:
from cogdl.attack.modification import FGA
n_mod_ratio = 0.1
attack = FGA(int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio),
             device=device)
graph_attack = attack.attack(model, graph)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))

 10%|▉         | 260/2680 [00:37<05:48,  6.94it/s]


FGA attack finished. 260 edges were flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10296], edge_attr=[10296])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [11]:
from cogdl.attack.modification import FLIP
n_mod_ratio = 0.01
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
attacks = {
    "degree_flip": FLIP(n_edge_mod, flip_type="deg", mode="descend", device=device),   # degree flipping
    "eigen_flip": FLIP(n_edge_mod, flip_type="eigen", mode="descend", device=device), # eigen flipping
    "betweenness_flip": FLIP(n_edge_mod, flip_type="bet", mode="ascend", device=device)     # betweenness flipping
}
for key, attack in attacks.items():
    graph_attack = attack.attack(graph)
    print(graph_attack)
    test_score = evaluate(model, 
                          graph_attack,
                          mask=test_mask,
                          device=device)
    print("After {} attack, test score of surrogate model: {:.4f}".format(key, test_score))
    test_score = evaluate(model_target, 
                          graph_attack,
                          mask=test_mask,
                          device=device)
    print("After {} attack, test score of target model: {:.4f}".format(key, test_score))

  1%|          | 26/2604 [00:00<00:00, 3949.44it/s]


FLIP attack finished. 26 edges were flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10244], edge_attr=[10244])
After degree_flip attack, test score: 0.6803
After attack, test score of target model: 0.6045


  1%|          | 26/2604 [00:00<00:00, 4217.50it/s]

FLIP attack finished. 26 edges were flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10244], edge_attr=[10244])
After eigen_flip attack, test score: 0.6853
After attack, test score of target model: 0.6045


In [7]:
from cogdl.attack.modification import RAND
n_mod_ratio = 0.1
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
attack = RAND(n_edge_mod, device=device)
graph_attack = attack.attack(graph)
print(graph_attack)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

 10%|▉         | 260/2604 [00:00<00:00, 6407.51it/s]

RAND attack finished. 260 edges were randomly flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 9788], edge_attr=[9788])
After attack, test score of surrogate model: 0.5609
After attack, test score of target model: 0.5808


In [8]:
from cogdl.attack.modification import NEA
n_mod_ratio = 0.1
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
attack = NEA(n_edge_mod, device=device)
graph_attack = attack.attack(graph)
print(graph_attack)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

 10%|▉         | 260/2604 [00:00<00:00, 7892.87it/s]

NEA attack finished. 260 edges were flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 9860], edge_attr=[9860])
After attack, test score of surrogate model: 0.5510
After attack, test score of target model: 0.5970


In [14]:
from cogdl.attack.modification import STACK
n_mod_ratio = 0.01
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
attack = STACK(n_edge_mod, device=device)
graph_attack = attack.attack(graph)
print(graph_attack)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

  1%|          | 26/2604 [00:00<00:00, 7016.14it/s]

STACK attack finished. 26 edges were flipped.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10248], edge_attr=[10248])
After attack, test score: 0.6729
After attack, test score of target model: 0.5983


In [17]:
from cogdl.attack.modification import PGD
epsilon = 0.1
n_epoch = 500
n_mod_ratio = 0.01
n_node_mod = int(graph.y.shape[0] * n_mod_ratio)
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
feat_lim_min = 0.0
feat_lim_max = 1.0
early_stop_patience = 200
attack = PGD(epsilon,
             n_epoch,
             n_node_mod,
             n_edge_mod,
             feat_lim_min,
             feat_lim_max,
             early_stop_patience=early_stop_patience,
             device=device)
graph_attack = attack.attack(model, graph)
print(graph_attack)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

  1%|          | 28/2604 [00:00<00:00, 3242.24it/s]
Epoch 499, Loss: 1.8289, Surrogate test score: 0.6194: 100%|██████████| 500/500 [00:07<00:00, 63.30it/s]


Surrogate test score: 0.6194
Attack runtime: 7.9239.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10244], edge_attr=[10244])
After attack, test score of surrogate model: 0.4614
After attack, test score of target model: 0.4366


In [5]:
from cogdl.attack.modification import PRBCD
device="cpu"
epsilon = 0.3
n_epoch = 500
n_mod_ratio = 0.01
n_node_mod = int(graph.y.shape[0] * n_mod_ratio)
n_edge_mod = int(graph.to_scipy_csr()[test_mask.cpu()].getnnz() * n_mod_ratio)
feat_lim_min = 0.0
feat_lim_max = 1.0
early_stop_patience = 200
early_stop_epsilon = 0.001
attack = PRBCD(epsilon,
              n_epoch,
              n_node_mod,
              n_edge_mod,
              feat_lim_min,
              feat_lim_max,
              early_stop_patience=early_stop_patience,
              early_stop_epsilon=early_stop_epsilon,
              device=device)
graph_attack = attack.attack(model, graph)
print(graph_attack)
test_score = evaluate(model, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of surrogate model: {:.4f}".format(test_score))
test_score = evaluate(model_target, 
                      graph_attack,
                      mask=test_mask,
                      device=device)
print("After attack, test score of target model: {:.4f}".format(test_score))

/home/xu-ll18/research/cogdl/cogdl/attack/modification/prbcd.py:440: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  + (n - row_idx) * ((n - row_idx) - 1) // 2
Epoch 499, Loss: 0.0076, Surrogate test score: 0.4366: 100%|██████████| 500/500 [25:30<00:00,  3.06s/it]


Surrogate test score: inf
Loading search space of epoch -inf

Attack runtime: 1541.8050.
Graph(x=[2680, 302], y=[2680], edge_index=[2, 10346], edge_attr=[10346])
After attack, test score of surrogate model: 0.5075
After attack, test score of target model: 0.3221
